<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa González
- Nombre de alumno 2: Benjamín Angulo


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [64]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_17180\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

5.- Se obtiene un MAE de 13546.49. En este contexto, un MAE menor implica que las predicciones están más cerca de los valores reales de demanda, o bien, implica una mejor precisión del modelo.

6.- Con XGBReressor, el MAE obtenido es de 2391.15. Es decir, en este caso el MAE es mejor y las predicciones están bastante más cerca de los valores reales de demanda.

In [91]:
# Inserte su código acá

# 1. separar datos en conjuntos train, val, test
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size = 0.1, random_state = 42)
train_data, val_data = train_test_split(train_data, test_size = 0.2222, random_state = 42)

X_train, y_train = train_data.drop(columns=['quantity']), train_data['quantity']
X_val, y_val = val_data.drop(columns=['quantity']), val_data['quantity']
X_test, y_test = test_data.drop(columns=['quantity']), test_data['quantity']

# 2. FuctionTransformer para extraer día, mes y año
from sklearn.preprocessing import FunctionTransformer

def extract_date_features(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df

date_transformer = FunctionTransformer(extract_date_features)

# 3. ColumnTransformer para procesar datos numéricos y categóricos
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_features = ['day', 'month', 'year', 'city', 'shop', 'brand', 'container', 'capacity']  # se considera que date ya fue transformado, capacity lo dejamos acá porque solo puede ser '500ml', '1.5lt', '330ml'
numerical_features = ['lat', 'long', 'pop', 'price']

col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# 4. guardar pasos anteriores en un pipeline incluyendo DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

dummy_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('regressor', DummyRegressor(strategy='mean'))
])

# 5. entrenar el pipeline anterior y reportar la métrica `mean_absolute_error` sobre los datos de validación
from sklearn.metrics import mean_absolute_error

dummy_pipeline.fit(X_train, y_train)

val_predictions = dummy_pipeline.predict(X_val)
mae_dummy = mean_absolute_error(y_val, val_predictions)
print(f'MAE del DummyRegressor: {mae_dummy}')

# 6. volver a entrenar pipeline con XGBRegressor y reportar la métrica `mean_absolute_error` sobre los datos de validación
from xgboost import XGBRegressor

xgb_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('regressor', XGBRegressor(objective='reg:squarederror'))
])

xgb_pipeline.fit(X_train, y_train)
val_predictions_xgb = xgb_pipeline.predict(X_val)
mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
print(f'MAE del XGBRegressor: {mae_xgb}')

MAE del DummyRegressor: 13546.494391911923
MAE del XGBRegressor: 2391.1526059023413


In [18]:
# 7. guardar ambos modelos
import joblib

joblib.dump(dummy_pipeline, 'dummy_pipeline.pkl')
joblib.dump(xgb_pipeline, 'xgb_pipeline.pkl')

['xgb_pipeline.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




3.- El MAE obtenido en este caso es de 2519.22. Dado que este valor es un poco mayor que el obtenido anteriormente con xgboost sin la relación monótona, nuestro amigo nos sopló mal, pues nuestro modelo ahora tiene un error más grande.

In [118]:
X_train, y_train = train_data.drop(columns=['quantity']), train_data['quantity']
X_val, y_val = val_data.drop(columns=['quantity']), val_data['quantity']
X_test, y_test = test_data.drop(columns=['quantity']), test_data['quantity']


# 1. entrenar nuevo pipeline con las restricciones de monotonicidad y que mantiene formato pandas hasta antes del step de entrenamiento
def to_dataframe(X):
    return pd.DataFrame(X.toarray(), columns=col_transformer.get_feature_names_out())

pandas_transformer = FunctionTransformer(to_dataframe)

xgb_pipeline_monotone = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('pandas_transformer', pandas_transformer),
    ('regressor', XGBRegressor(objective='reg:squarederror', monotone_constraints={'num__price': -1}))
])

xgb_pipeline_monotone.fit(X_train, y_train)

# 2. reportar MAE sobre conjunto de validación
val_predictions_xgb_monotone = xgb_pipeline_monotone.predict(X_val)
mae_xgb_monotone = mean_absolute_error(y_val, val_predictions_xgb_monotone)
print(f'MAE del XGBRegressor con restricción de monotonicidad: {mae_xgb_monotone}')

MAE del XGBRegressor con restricción de monotonicidad: 2519.222899080682


## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

A continuación se explica cada hiperparámetro:

1. `learning_rate:` Controla la tasa de aprendizaje del modelo. Un valor más bajo puede hacer que el modelo aprenda más lentamente, pero podría mejorar la precisión. Los valores entre 0.001 y 0.1 son comunes y razonables. Un learning_rate muy bajo (cerca de 0.001) permite ajustes muy precisos, pero puede requerir más iteraciones, mientras que valores cercanos a 0.1 permiten un aprendizaje más rápido. Valores fuera de este rango podrían resultar en un aprendizaje demasiado lento o demasiado rápido, causando overfitting o underfitting, respectivamente.

2. `n_estimators:` Número de árboles en el modelo. Más árboles pueden aumentar la capacidad del modelo, pero también el riesgo de overfitting. El rango de 50 a 1000 es apropiado, pues menos de 50 árboles pueden no ser suficientes para captar la complejidad del modelo, mientras que más de 1000 pueden causar overfitting y aumentar demasiado el tiempo de cómputo sin que esto implique mejores resultados.

3. `max_depth:` Profundidad máxima de los árboles. Controla la complejidad de cada árbol. Un rango de 3 a 10 es razonable, ya que valores por debajo de 3 pueden hacer que los árboles sean demasiado simples para captar patrones complejos, mientras valores superiores a 10 pueden hacer que los árboles se vuelvan demasiado complejos y produzcan overfitting.

4. `max_leaves:` Número máximo de hojas por árbol. Permite controlar la complejidad de cada árbol, pero de una manera un poco más directa que max_depth. El rango de 0 a 100 es amplio y flexible, permitir hasta 100 hojas proporciona suficiente capacidad para captar patrones complejos sin sobreajustar en la mayoría de los casos.

5. `min_child_weight:` Peso mínimo que debe tener una hoja. Ayuda a evitar que el modelo cree hojas con muy pocos datos. El rango de 1 a 5 es adecuado, ya que un valor más bajo (1) permite mayor flexibilidad para crear hojas pequeñas, mientras que un valor más alto (hasta 5) fuerza al modelo a tener hojas con mayor número de ejemplos, lo que puede ayudar a evitar el overfitting.

6. `reg_alpha:` Regularización L1. Ayuda a hacer que el modelo sea más sencillo y puede ayudar a prevenir el overfitting. Un rango de 0 a 1 es común y sensato, ya que valores cercanos a 0 significan poca regularización, mientras que valores cercanos a 1 implican una fuerte regularización. Esto permite explorar desde ningún efecto de regularización hasta una regularización fuerte.

7. `reg_lambda:` Regularización L2. Similar a reg_alpha, pero penaliza el tamaño de los coeficientes del modelo. Similar a reg_alpha, un rango de 0 a 1 es apropiado, ya que permite explorar desde ningún efecto de regularización hasta una regularización fuerte.

8. `min_frequency:` n el OneHotEncoder, este parámetro ayuda a ignorar categorías poco frecuentes para evitar ruido en los datos categóricos. El rango de 0.0 a 1.0 es correcto, ya que cubre desde no ignorar ninguna categoría (0.0) hasta ignorar todas las categorías (1.0). 

En cuanto al número de trials, se tienen 196, y con los mejores hiperparámetros el MAE es de 1922.33. Considerando que el MAE del mejor modelo anterior es de 2391.15, se tiene que los resultados luego de hiperparámetros mejoran bastante.

Lo anterior se puede deber a una combinación de ajustes precisos en los distintos hiperparámetros abordados. La metodología de búsqueda bayesiana, sumada a los rangos de valores escogidos que nos permiten la exploración suficiente del espacio de hiperparámetros, encuentran una configuración que optimiza el rendimiento del modelo.

In [120]:
# Inserte su código acá
import optuna
from optuna.samplers import TPESampler

# separación de datos
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2222, random_state=42)

X_train, y_train = train_data.drop(columns=['quantity']), train_data['quantity']
X_val, y_val = val_data.drop(columns=['quantity']), val_data['quantity']
X_test, y_test = test_data.drop(columns=['quantity']), test_data['quantity']

def objective(trial):

    # hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # pipeline
    date_transformer = FunctionTransformer(extract_date_features)
    col_transformer = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(min_frequency = min_frequency), categorical_features)
        ]
    )

    xgb_pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('col_transformer', col_transformer),
        ('regressor', XGBRegressor(
            objective = 'reg:squarederror',
            learning_rate = learning_rate,
            n_estimators = n_estimators,
            max_depth = max_depth,
            max_leaves = max_leaves,
            min_child_weight = min_child_weight,
            reg_alpha = reg_alpha,
            reg_lambda = reg_lambda,
            random_state = 42
        ))
    ])

    xgb_pipeline.fit(X_train, y_train)
    val_predictions_xgb = xgb_pipeline.predict(X_val)
    mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
    
    return mae_xgb

# estudio de optuna
study = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = 42))
study.optimize(objective, timeout=300)  # 5 minutos

# mejores hiperparámetros
best_params = study.best_params
best_trial = study.best_trial

print(f'Número de trials: {len(study.trials)}')
print(f'MAE del mejor trial: {best_trial.value}')
print(f'Mejores hiperparámetros: {best_params}')

# entrenar el modelo final con los mejores hiperparámetros
final_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('col_transformer', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(min_frequency = best_params['min_frequency']), categorical_features)
        ]
    )),
    ('regressor', XGBRegressor(
        objective='reg:squarederror',
        learning_rate = best_params['learning_rate'],
        n_estimators = best_params['n_estimators'],
        max_depth = best_params['max_depth'],
        max_leaves = best_params['max_leaves'],
        min_child_weight = best_params['min_child_weight'],
        reg_alpha = best_params['reg_alpha'],
        reg_lambda = best_params['reg_lambda'],
        random_state = 42
    ))
])

final_pipeline.fit(X_train, y_train)

# guardar modelo final
joblib.dump(final_pipeline, 'xgb_pipeline_optimized.pkl')

# evaluar en el conjunto de validación
val_predictions_final = final_pipeline.predict(X_val)
mae_final = mean_absolute_error(y_val, val_predictions_final)
print(f'MAE final del modelo optimizado: {mae_final}')

[I 2024-05-29 21:37:29,731] A new study created in memory with name: no-name-7f555541-1457-4c45-bea1-8ed6fbd0eab1
[I 2024-05-29 21:37:31,324] Trial 0 finished with value: 9749.174034768506 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 9749.174034768506.
[I 2024-05-29 21:37:31,915] Trial 1 finished with value: 5757.241405136549 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 5757.241405136549.
[I 2024-05-29 21:37:32,689] Trial 2 finished with value: 8966.786244812625 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_depth

Número de trials: 196
MAE del mejor trial: 1922.3255411374496
Mejores hiperparámetros: {'learning_rate': 0.06352493662365062, 'n_estimators': 938, 'max_depth': 10, 'max_leaves': 96, 'min_child_weight': 3, 'reg_alpha': 0.5076832949274352, 'reg_lambda': 0.7313463054648354, 'min_frequency': 5.177230606046718e-05}
MAE final del modelo optimizado: 1922.3255411374496


## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>